### Notebook for the testing of `qtr` in the heart with and without failure. 
- **Developed by** Carlos Talavera-López Ph.D
- **Institute of AI for Health - HelmholtzZentrum münchen**
- v210830

The idea is to use `qtr` to evaluate the biological differences between protocols used for the human heart project.  

### Install the latest version of `qtr`

In [1]:
!pip install -e git+https://github.com/theislab/scarches.git@sergey#egg=scarches
!pip install --user scikit-misc

Obtaining scarches from git+https://github.com/theislab/scarches.git@sergey#egg=scarches
  Updating ./src/scarches clone (to revision sergey)
  Running command git fetch -q --tags
  Running command git reset --hard -q b05231261842951f05db3cf78247344e3d05d4ce
  Attempting uninstall: scarches
    Found existing installation: scArches 0.3.5
    Can't uninstall 'scArches'. No files were found to uninstall.
  Running setup.py develop for scarches


### Load required modules 

In [2]:
import torch
import warnings
import numpy as np
import scanpy as sc
import scarches as sca
import matplotlib.pyplot as plt

### Set up working environment

In [3]:
warnings.simplefilter(action = 'ignore')
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 200, color_map = 'RdPu', dpi_save = 300, vector_friendly = True, format = 'svg')
torch.set_printoptions(precision = 3, sci_mode = False, edgeitems = 7)

The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata     0.7.6
scanpy      1.8.1
sinfo       0.3.4
-----
PIL                         7.1.2
astor                       0.8.1
astunparse                  1.6.3
attr                        21.2.0
beta_ufunc                  NA
binom_ufunc                 NA
bottleneck                  1.3.2
cached_property             1.5.2
certifi                     2021.05.30
cffi                        1.14.6
chardet                     3.0.4
cloudpickle                 1.3.0
colorama                    0.4.4
cycler

- Check GPU type assigned by `colab`

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Sep  9 12:24:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Read in single nuclei data from `GDrive`

- Data is anonymised as it only contains raw counts and non-identifiable covariates.  

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Read LV object from the human heart cell atlas. 

- The data was downloaded from the Heart Cell Atlas project [website](https://www.heartcellatlas.org). 
- The left ventricle (LV) data was selected for this analysis to make it more lightweight and focused. 

In [6]:
heart_LV = sc.read_h5ad('/content/gdrive/My Drive/data/reference/single_cell/qtr_heart/heart_LV.10Xsn-iCell8.healthy-diseased.ctl210830.raw.h5ad')
heart_LV

AnnData object with n_obs × n_vars = 84748 × 15224
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used', 'Cells_Nuclei', 'combined', 'Barcode', 'Type', 'Individual', 'Age', 'Gender', 'state'
    var: 'gene_ids-Harvard-Nuclei-full-healthy', 'feature_types-Harvard-Nuclei-full-healthy', 'gene_ids-Sanger-Nuclei-full-healthy', 'feature_types-Sanger-Nuclei-full-healthy', 'gene_ids-Sanger-Cells-full-healthy', 'feature_types-Sanger-Cells-full-healthy', 'gene_ids-Sanger-CD45-full-healthy', 'feature_types-Sanger-CD45-full-healthy', 'n_cells-myeloid-healthy', 'n_counts-myeloid-healthy'
    layers: 'counts'

In [7]:
healthy_LV = heart_LV[heart_LV.obs['state'].isin(['healthy'])]
healthy_LV

View of AnnData object with n_obs × n_vars = 82806 × 15224
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used', 'Cells_Nuclei', 'combined', 'Barcode', 'Type', 'Individual', 'Age', 'Gender', 'state'
    var: 'gene_ids-Harvard-Nuclei-full-healthy', 'feature_types-Harvard-Nuclei-full-healthy', 'gene_ids-Sanger-Nuclei-full-healthy', 'feature_types-Sanger-Nuclei-full-healthy', 'gene_ids-Sanger-Cells-full-healthy', 'feature_types-Sanger-Cells-full-healthy', 'gene_ids-Sanger-CD45-full-healthy', 'feature_types-Sanger-CD45-full-healthy', 'n_cells-myeloid-healthy', 'n_counts-myeloid-healthy'
    layers: 'counts'

In [8]:
healthy_LV.X

<82806x15224 sparse matrix of type '<class 'numpy.float32'>'
	with 93697072 stored elements in Compressed Sparse Row format>

### Add module genes

- Here I have selected the C2 modules from the repo, and I have added the C3 Immune-related modules from GSEA. 

In [9]:
path_gmt = '/content/gdrive/My Drive/data/reference/modules/'
#sca.add_annotations(LV_nuclei, 
#                    [path_gmt + 'c2.cp.reactome.v4.0.symbols.gmt', path_gmt + 'c7.all.v7.4.symbols.gmt', path_gmt + 'c3.all.v7.4.symbols.gmt'], 
#                    min_genes = 20, clean = True)
sca.add_annotations(healthy_LV, 
                    [path_gmt + 'c2.cp.reactome.v4.0.symbols.gmt', path_gmt + 'c3.all.v7.4.symbols.gmt'], 
                    min_genes = 20, clean = False)

healthy_LV._inplace_subset_var(healthy_LV.varm['I'].sum(1) > 0)

### Process module genes that match input
- Although the processed data comes with 7K HVGs already labelled in the `adata` object, I selected 2K HVGs to make things faster. 

In [10]:
sc.pp.highly_variable_genes(
    healthy_LV,
    flavor = "seurat_v3",
    n_top_genes = 3000,
    layer = "counts",
    batch_key = "combined",
    subset = False,
    span = 1
)
healthy_LV

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


AnnData object with n_obs × n_vars = 82806 × 14784
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used', 'Cells_Nuclei', 'combined', 'Barcode', 'Type', 'Individual', 'Age', 'Gender', 'state'
    var: 'gene_ids-Harvard-Nuclei-full-healthy', 'feature_types-Harvard-Nuclei-full-healthy', 'gene_ids-Sanger-Nuclei-full-healthy', 'feature_types-Sanger-Nuclei-full-healthy', 'gene_ids-Sanger-Cells-full-healthy', 'feature_types-Sanger-Cells-full-healthy', 'gene_ids-Sanger-CD45-full-healthy', 'feature_types-Sanger-CD45-full-healthy', 'n_cells-myeloid-healthy', 'n_counts-myeloid-healthy', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'terms', 'hvg'
    varm: 'I'
    layers: 'counts'

In [11]:
select_terms = healthy_LV.varm['I'].sum(0) > 20
healthy_LV.uns['terms'] = np.array(healthy_LV.uns['terms'])[select_terms].tolist()
healthy_LV.varm['I'] = healthy_LV.varm['I'][:, select_terms]
healthy_LV._inplace_subset_var(healthy_LV.varm['I'].sum(1) > 0)
healthy_LV.X -= healthy_LV.X.mean(0)
healthy_LV.X = healthy_LV.layers["counts"].copy()

### Create TRVAE model and train it on reference dataset

In [12]:
healthy_LV.X = healthy_LV.X.toarray()

In [13]:
intr_cvae = sca.models.TRVAE(
    adata = healthy_LV,
    condition_key = 'combined',
    hidden_layer_sizes = [1000, 600, 600],
    use_mmd = False,
    recon_loss = 'nb',
    mask = healthy_LV.varm['I'].T,
    use_decoder_relu = False,
    mmd_instead_kl = False
)


INITIALIZING NEW NETWORK..............
Encoder Architecture:
	Input Layer in, out and cond: 14784 1000 14
	Hidden Layer 1 in/out: 1000 600
	Hidden Layer 2 in/out: 600 600
	Mean/Var Layer in/out: 600 3782
Decoder Architecture:
	Masked linear layer in, out and cond:  3782 14784 14


- Set up training parameters 

In [14]:
ALPHA = 0.7

In [15]:
OMEGA = torch.Tensor(healthy_LV.varm['I'].sum(0))
OMEGA/=OMEGA.max()
OMEGA*=10

In [16]:
OMEGA = None

In [17]:
LR = 0.0001

In [18]:
early_stopping_kwargs = {
    "early_stopping_metric": "val_unweighted_loss",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
}

In [19]:
intr_cvae.train(
    n_epochs = 500, 
    alpha_epoch_anneal = 200, 
    alpha = ALPHA, 
    omega = OMEGA,
    alpha_kl = 0.001,
    weight_decay = 0., 
    early_stopping_kwargs = early_stopping_kwargs,
    use_early_stopping = True,
    seed = 1712,
    print_n_deactive = False)

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


 |█████████-----------| 48.0%  - epoch_loss: 3275.96 - epoch_recon_loss: 3251.09 - epoch_kl_loss: 24867.86 - val_loss: 3357.44 - val_recon_loss: 3341.85 - val_kl_loss: 15589.79
ADJUSTED LR
 |█████████-----------| 49.4%  - epoch_loss: 3271.55 - epoch_recon_loss: 3245.16 - epoch_kl_loss: 26384.85 - val_loss: 3347.72 - val_recon_loss: 3330.72 - val_kl_loss: 16998.86
Stopping early: no improvement of more than 0 nats in 20 epochs
If the early stopping criterion is too strong, please instantiate it with different parameters in the train method.
Saving best state of network...
Best State was in Epoch 225


### Checl terms in results

In [20]:
inactive_idx = ~(intr_cvae.model.decoder.L0.expr_L.weight.data.norm(p = 2, dim = 0) > 0).cpu().numpy()

In [21]:
print('Inactive terms:')
[term for i, term in enumerate(healthy_LV.uns['terms']) if inactive_idx[i]]

Inactive terms:


['REACTOME_ACTIVATION_OF_THE_PRE_REPLICATIVE_COMPLEX',
 'REACTOME_G0_AND_EARLY_G1',
 'REACTOME_SIGNALLING_TO_RAS',
 'REACTOME_KERATAN_SULFATE_BIOSYNTHESIS',
 'REACTOME_ERK_MAPK_TARGETS',
 'REACTOME_GENERATION_OF_SECOND_MESSENGER_MOLECULES',
 'REACTOME_CHEMOKINE_RECEPTORS_BIND_CHEMOKINES',
 'REACTOME_GLUCAGON_TYPE_LIGAND_RECEPTORS',
 'REACTOME_ELONGATION_ARREST_AND_RECOVERY',
 'REACTOME_SIGNAL_AMPLIFICATION',
 'REACTOME_METAL_ION_SLC_TRANSPORTERS',
 'REACTOME_TRANSPORT_OF_VITAMINS_NUCLEOSIDES_AND_RELATED_MOLECULES',
 'MIR296_3P',
 'MIR1914_3P',
 'MIR4423_5P',
 'MIR6787_5P',
 'XPO1_TARGET_GENES',
 'STAT3_01']

### Visualise manifold

In [ ]:
MEAN = False
healthy_LV.obsm['X_cvae'] = intr_cvae.get_latent(mean = MEAN)[:, ~inactive_idx]

In [ ]:
sc.pp.neighbors(healthy_LV, use_rep = "X_cvae", n_neighbors = 20, metric = 'minkowski')
sc.tl.umap(healthy_LV, min_dist = 0.3, spread = 2, random_state = 1712)
sc.pl.umap(healthy_LV, frameon = False, color = ['cell_source', 'donor', 'cell_states', 'state'], size = 0.6, legend_fontsize = 5)

### Train on damaged heart data

In [ ]:
damaged_LV = heart_LV[heart_LV.obs['state'].isin(['damaged'])]
damaged_LV

In [ ]:
damaged_LV = damaged_LV[:, healthy_LV.var_names]
damaged_LV = damaged_LV.copy()
damaged_LV

In [ ]:
damaged_LV.X = damaged_LV.X.toarray()

In [ ]:
q_intr_cvae = sca.models.TRVAE.load_query_data(damaged_LV, intr_cvae)

In [ ]:
q_intr_cvae.train(n_epochs = 500, alpha_epoch_anneal = 200, weight_decay = 0., alpha_kl = 0.1, seed = 1786)

### Visualise manifold for `damaged_LV`

In [ ]:
q_intr_cvae.adata

In [ ]:
damaged_LV

In [ ]:
damaged_LV.obsm['X_cvae'] = q_intr_cvae.get_latent(mean = MEAN)[:, ~inactive_idx]

In [ ]:
sc.pp.neighbors(damaged_LV, use_rep = "X_cvae", n_neighbors = 20, metric = 'minkowski')
sc.tl.umap(damaged_LV, min_dist = 0.3, spread = 2, random_state = 1712)
sc.pl.umap(damaged_LV, frameon = False, color = ['cell_source', 'donor', 'cell_states'], size = 0.6, legend_fontsize = 5)

### Subset terms of interest

In [ ]:
terms = damaged_LV.uns['terms']
terms

In [ ]:
idx = [terms.index(term) for term in ['CYTOKINE_SIGNALING_IN_IMMUNE_S', 'INNATE_IMMUNE_SYSTEM', 'SIGNALING_BY_NOTCH']]
latents = q_intr_cvae.get_latent(mean = MEAN)[:, idx]

### Explore terms in diseased data

In [ ]:
damaged_LV.obs['CYTOKINE_SIGNALING_IN_IMMUNE_S'] = latents[:, 0]
damaged_LV.obs['INNATE_IMMUNE_SYSTEM'] = latents[:, 1]
damaged_LV.obs['SIGNALING_BY_NOTCH'] = latents[:, 2]

In [ ]:
sc.pl.scatter(damaged_LV, x = 'INNATE_IMMUNE_SYSTEM', y = 'SIGNALING_BY_NOTCH', color = 'cell_source', size = 22)

In [ ]:
latents = intr_cvae.get_latent(mean = MEAN)[:, idx]

In [ ]:
damaged_LV.obs['INNATE_IMMUNE_SYSTEM'] = latents[:, 0]
damaged_LV.obs['SIGNALING_BY_NOTCH'] = latents[:, 1]

In [ ]:
sc.pl.scatter(damaged_LV, x = 'INNATE_IMMUNE_SYSTEM', y = 'SIGNALING_BY_NOTCH', color='cell_source', size = 22)

### Display joint representation

In [ ]:
heart = sc.AnnData.concatenate(healthy_LV, damaged_LV, batch_key = 'batch_join')
heart

In [ ]:
heart.obsm['X_cvae'] = q_intr_cvae.get_latent(heart.X, heart.obs['combined'].tolist(), mean = MEAN)[:, ~inactive_idx]
sc.pp.neighbors(heart, use_rep = "X_cvae", n_neighbors = 20, metric = 'minkowski')
sc.tl.umap(heart, min_dist = 0.3, spread = 2, random_state = 1712)
sc.pl.umap(heart, frameon = False, color = ['cell_source', 'donor', 'cell_states'], size = 0.6, legend_fontsize = 5)